# Nanoparticle Analysis Tools
A set of functions to compute descriptors designed to tease apart the properties of the simulated nanoparticles. Two of which are shape based, two of which are thermodynamic properties. All are independent of the shape and size of the drug molecules. The properties are:
 1. Packing fraction
   * The ratio of of the total atomic volume of the drugs and dye to the solvent accessible solvent volume.
 2. Compressibility
   * Quantifies the sensitivity of the nanoparticle with respect to changes in pressure
 3. Morphability
   * Quantifies the sensitivity of the surface area of the nanoparticle with respect to changes in the surface tension.
 4. Sphericity
   * Measures how close to spherical the overall shape of the nanoparticle is.

# 1. Packing fraction
Requires functions to calculate the atomic volume of molecule, based on the van der Waals volume of each atom, and a function to compute the volume of a nanoparticle. Defined as 

$$ pf = \frac{V_{atoms}}{V_{nanoparticle}} $$


## Volume of atoms

In [1]:
def ResidueVolume(traj,resname):
    '''
    Calculate the volume of the atomic constituents for residues in a trajectory.
    
    Parameters
    ----------
    traj  : mdtraj trajectory object
      Trajectory containing the residue of interest
    resname : string 
      the name of the residue

    Returns
    -------
    residue_vol : float
      the vdW volume of a single residue in nm^3
    total_vol : float
      the total volume of all matching residues in nm^3
    """
    
    
    '''
    residues = [res for res in traj.topology.residues if res.name==resname]
    if len(residues) == 0:
        print "No residues called '%s' found in trajectory."
        return
    else:
        numres = len(residues)
        
    atoms = [atm.element for atm in residues[0].atoms]
    radii = np.zeros(len(atoms))
    
    for i in range(len(atoms)):
        radii[i] = atoms[i].radius
        #print atm.symbol, radii

    residue_vol =  np.sum(np.pi*(4.0/3.0)*(radii**3))
    total_vol = residue_vol*numres
    
    return residue_vol, total_vol  

## Volume of nanoparticle
A more complicated series of functions are required to evaluate this quantity. My most successful function is based on `calc_density.py` from ProtoMS 3. The performace of the functions below are evaluated in the Jupyter notebook called `VolumeEstimation`.

In [2]:
def init_grid(xyz,spacing,padding) :
    """
    Initialize a grid based on a list of x,y,z coordinates. Written by Sam Genheden.

    Parameters
    ----------
    xyz  : Numpy array 
      Cartesian coordinates that should be covered by the grid
    spacing : float 
      the grid spacing
    padding : float 
      the space to add to minimum extent of the coordinates

    Returns
    -------
    Numpy array
      the grid 
    list of Numpy arrays
      the edges of the grid
    """
  
    origin  = np.floor(xyz.min(axis=0))-padding
    tr      = np.ceil(xyz.max(axis=0))+padding
    length  = tr-origin
    shape  =  np.array([int(l/spacing + 0.5) + 1 for l in length],dtype=int)
    grid    = np.zeros(shape)
    edges  = [np.linspace(origin[i],tr[i],shape[i]) for i in range(3)]
    return grid,edges

def _voxel(coord,edges) :
    """
    Wrapper for the numpy digitize function to return the grid coordinates. Written by Sam Genheden.
    """
    return np.array([np.digitize(coord,edges[i])[i] for i in range(3)],dtype=int)

def _fill_sphere(coord,grid,edges,spacing,radius) :
    """
    Fill a grid using spherical smoothing

    Parameters
    ----------
    coord : Numpy array
      the Cartesian coordinates to put on the grid
    grid  : Numpy array
      the 3D grid. Will be modified
    edges : list of Numpy array
      the edges of the grid
    spacing : float 
      the grid spacing
    radius  : float 
      the radius of the smoothing
    """
    # Maximum coordinate
    maxxyz = np.minimum(coord + radius,np.array([edges[0][-1],edges[1][-1],edges[2][-1]]))
    i = 1
    # Iterate over the sphere
    rad2 = radius**2
    x = max(coord[0] - radius,edges[0][0])
    while x <= maxxyz[0] :
        y = max(coord[1] - radius,edges[1][0])
        while y <= maxxyz[1] :
            z = max(coord[2] - radius,edges[2][0])
            while z <= maxxyz[2] :
            # Check if we are on the sphere
                r2 = (x-coord[0])**2+(y-coord[1])**2+(z-coord[2])**2
                if r2 <= rad2 :
                # Increase grid with one
                    v = _voxel(np.array([x,y,z]),edges)
                    grid[v[0],v[1],v[2]] = grid[v[0],v[1],v[2]] + 1
                z = z + spacing
            y = y + spacing
        x = x + spacing
        #print x
        
    
def SolvAccVol(traj, residues, frame, solvent_rad=0.14, spacing=0.05) :
    """
    Calculate the solvent accessible volume of a set of residues using spherical smoothing

    Parameters
    ----------
    traj : Mdtraj trejectory object
      the trajectory containing the residues of interest
    residues  : list or tuple of strings
      the names of the residues whose volume will be calculated
    frame : integer
      the frame from which the positions will be taken from
    solvent_rad : float 
      the radius of the solvent in nm. Arithmetic mean with residue will be used for volume calculation.
      If zero, only the vdW radii of the atoms will be used.
    spacing : float 
      the grid spacing in nanometers  
      
    Returns:
    -------
    volume : float
      the solvent accessible volume of the listed residues
    """
    
    atmids = [atom.index for atom in traj.topology.atoms if ((atom.residue.name in residues))]
    radii = [atom.element.radius for atom in traj.topology.atoms if ((atom.residue.name in residues))]
    if solvent_rad > 0.0:
        watrads = np.repeat(solvent_rad,len(radii))
        radii = 0.5*(radii + watrads)
    
    xyz = traj.xyz[frame]
    xyz = xyz[atmids,:]
    
    grid,edges = init_grid(xyz,spacing,0)
    
    for i in range(len(atmids)):
        _fill_sphere(xyz[i],grid,edges,spacing,radii[i])
    
    volume = np.sum(grid >= 1)*(spacing**3)
    return volume

# 2. Compressibility
The thermal compressibility is a thermodynamic parameter that quantifies the variance of the volume of a system. It's related to the second derivative of the partition function with respect to volume. In short, the compressibility quantifies how susceptible a nanoparticle is to changes in pressure. 

Compressibility is defined as

$$ c = \frac{1}{k_B T}\frac{(\Delta V)^2}{V}  = -\frac{1}{V} \left(\frac{\delta V}{\delta p} \right) ,$$

where $V$ is the average volume, $(\Delta V)^2$ the variance, and $p$ is the pressure.

In [4]:
def compressibilty(traj,residues,skip=0,step=5,solvent_rad=0.14,spacing=0.05,temperature=300.0):
    """
    Calculate the thermal compressibility of atoms in a trajectory.

    Parameters
    ----------
    traj : Mdtraj trejectory object
      the trajectory containing the residues of interest
    skip  : integer
      the number of frames from the start of the simulation that will be discarded as equilibration
    step : integer
      the amount to increment each frame by for volume calculation. E.g. step=1, every frame from skip will be used, 
      if step=10, the compressibility will be evaluated from every 10th frame.
    solvent_rad : float 
      the radius of the solvent in nm. Arithmetic mean with residue will be used for volume calculation.
      If zero, only the vdW radii of the atoms will be used.
    spacing : float 
      the grid spacing (in nanometers) that is used for volume estimation.
    temperature: float
      the temperature of the simulation in kelvin
      
    Returns
    -------
    comp : float
      the thermal compressibility in kcal/mol
    mean_vol : float
      the mean solvent accessible of residues in nm^3
    """
    
    kT = 0.0019872041*temperature # kcal/mol
    indices = range(skip,traj.n_frames,step)
    volume = np.zeros(len(indices))
    for i in range(len(indices)):
        volume[i] = SolvAccVol(traj, residues, indices[i], solvent_rad, spacing) 
    mean_vol = volume.mean()
    var_vol = volume.var()
    comp = var_vol/mean_vol/kT
    return comp, mean_vol

# 3. Morphability
One can define the surface area analogue to the thermal compressibility by qunatifing the variance of the surface area. It would be related to the second derivative of the partition function with respect to the surface tension, and would quantify how easily a nanoparticle would change its surface area by changing the surface tension. 

I define morphability as

$$ m = \frac{1}{k_B T}\frac{(\Delta A)^2}{A}  = -\frac{1}{A} \left(\frac{\delta A}{\delta \gamma} \right) ,$$

where $A$ is the average surface area, $(\Delta A)^2$ the variance, and $\gamma$ is the surface tension.

In [8]:
def morphability(traj,residues,solvent_rad=0.14,temperature=300):
    """
    Calculate the thermal compressibility of atoms in a trajectory.

    Parameters
    ----------
    traj : Mdtraj trejectory object
      the trajectory containing the residues of interest
    residues : tuple or list of strings
      the names of the residues whose solvent accessible surface area will be calculated
    solvent_rad : float 
      the radius of the solvent in nm. Arithmetic mean with residue will be used for volume calculation.
      If zero, only the vdW radii of the atoms will be used.
    spacing : float 
      the grid spacing (in nanometers) that is used for volume estimation.
      
    Returns
    -------
    comp : float
      the thermal compressibility in kcal/mol
    mean_vol : float
      the mean solvent accessible of residues in nm^3
    """
    
    kT = 0.0019872041*temperature # kcal/mol
    indices = range(skip,traj.n_frames,step)
    volume = np.zeros(len(indices))
    sasa = md.shrake_rupley(traj,probe_radius=solvent_rad,mode="residue")
    resids =[res.index for res in trajectory.topology.residues if res.name in target]
    total_sasa[:,resids].sum(axis=1)
    mean_sasa = total_sasa.mean()
    var_sasa = total_sasa.var()
    morph = var_sasa/mean_sasa/kT
    return morph, mean_sasa

# 4. Sphericity
Describes how close in shape a nanoparticle is to a sphere. Defined as

$$\Psi = \frac{\pi^{\frac{1}{3}}(6V)^{\frac{2}{3}}}{A},$$
    
where V is volume and A is surface area, $\Psi$ is a number between 0 and 1, where $\Psi=0$ is completely unlike a sphere, and  $\Psi=1$ when a nanoparticle is exactly a sphere.

In [7]:
def sphericity(V,A):
    return (np.pi)**(1.0/3.0)*((6*V)**(2.0/3.0))/A